#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18968 sha256=791df8cb29bec307fc51c5bef5cfe7c8d5fc76617ea373a729a859cc8a15ceac
  Stored in directory: /tmp/pip-ephem-wheel-cache-32_1tlzk/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


: 

## Load extra features

In [ ]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [ ]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-21
tomorrows_datestring:  2023-10-22


In [ ]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'
#json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.h5'

json_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.h5'
#json_file = "./daily_models/Boston311KerasNLP/20231014_234834_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231014_234834_Boston311KerasNLP.h5"
#json_file = "./daily_models/Boston311KerasNLP/20231015_004906_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231015_004906_Boston311KerasNLP.h5"

json_file = "./daily_models/Boston311KerasNLP/20231016_113838_Boston311KerasNLP.json"
model_file = "./daily_models/Boston311KerasNLP/20231016_113838_Boston311KerasNLP.h5"

#json_file = "./daily_models/Boston311KerasNLP/20231016_121447_Boston311KerasNLP.json"a
#model_file = "./daily_models/Boston311KerasNLP/20231016_121447_Boston311KerasNLP.h5"

#json_file = "./daily_models/Boston311KerasNLP/20231016_132122_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231016_132122_Boston311KerasNLP.h5"

json_file = "./daily_models/Boston311KerasNLP/20231020_181649_Boston311KerasNLP.json"
model_file = "./daily_models/Boston311KerasNLP/20231020_181649_Boston311KerasNLP.h5"

json_file = "./daily_models/Boston311KerasNLP/20231020_205853_Boston311KerasNLP.json"
model_file = "./daily_models/Boston311KerasNLP/20231020_205853_Boston311KerasNLP.h5"

json_file = "./daily_models/Boston311KerasNLP/20231020_214340_Boston311KerasNLP.json"
model_file = "./daily_models/Boston311KerasNLP/20231020_214340_Boston311KerasNLP.h5"


In [ ]:
import glob
import os

def get_latest_model_files(model_dir="./daily_models/Boston311KerasNLP/"):
    # Get all json and h5 files
    json_files = glob.glob(os.path.join(model_dir, "*.json"))
    h5_files = glob.glob(os.path.join(model_dir, "*.h5"))

    # Sort files by modification time
    json_files.sort(key=os.path.getmtime, reverse=True)
    h5_files.sort(key=os.path.getmtime, reverse=True)

    if json_files and h5_files:
        latest_json = json_files[0]
        latest_h5 = h5_files[0]
        return latest_json, latest_h5
    else:
        return None, None

# Use the function
json_file, model_file = get_latest_model_files()

if json_file and model_file:
    kerasNLP_model = Boston311KerasNLP()
    kerasNLP_model.load(json_file, model_file)
    kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


2023-10-21 00:47:00.365882: E tensorflow/compiler/xla/stream_executor/rocm/rocm_driver.cc:302] failed call to hipInit: HIP_ERROR_InvalidDevice
2023-10-21 00:47:00.365900: I tensorflow/compiler/xla/stream_executor/rocm/rocm_diagnostics.cc:111] retrieving ROCM diagnostic information for host: briarmossdesktop
2023-10-21 00:47:00.365905: I tensorflow/compiler/xla/stream_executor/rocm/rocm_diagnostics.cc:118] hostname: briarmossdesktop
2023-10-21 00:47:00.365985: I tensorflow/compiler/xla/stream_executor/rocm/rocm_diagnostics.cc:141] librocm reported version is: NOT_FOUND: was unable to find librocm.so DSO loaded into this program
2023-10-21 00:47:00.365990: I tensorflow/compiler/xla/stream_executor/rocm/rocm_diagnostics.cc:145] kernel reported version is: UNIMPLEMENTED: kernel reported driver version not implemented
2023-10-21 00:47:00.380111: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:00.380955: I tensorflow/core/common_runtime/gpu_f

In [ ]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [ ]:
data = kerasNLP_model.load_data( data=mydata, train_or_predict='predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [ ]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
2724585,101005114379,2023-10-16 19:57:00,2023-10-20 04:30:00,NaT,ONTIME,Open,,Work Hours-Loud Noise Complaints,Public Works Department,Highway Maintenance,...,50 Rockne Ave,2124.0,42.280310,-71.052071,0101000020E61000001C92F41F55C351C0CD88A82EE123...,Constituent Call,NaT,0,16,NaN
2724584,101005114377,2023-10-16 19:38:21,2023-10-18 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,10 Gladstone St,2128.0,42.388959,-71.000130,0101000020E61000004D094D2202C051C0FCB64E6CC931...,Citizens Connect App,NaT,0,1,NaN
2724583,101005114376,2023-10-16 19:33:03,2023-10-19 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,423 Bunker Hill St,2129.0,42.383040,-71.070560,0101000020E6100000081DD90F84C451C08C0180700731...,Citizens Connect App,NaT,0,2,NaN
2724582,101005114373,2023-10-16 19:21:32,2023-10-19 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,31 W Cedar St,2108.0,42.358733,-71.069771,0101000020E61000009DBCC32277C451C03AD29DF2EA2D...,Citizens Connect App,NaT,0,5,NaN
2724581,101005114370,2023-10-16 19:09:10,2023-10-18 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,27 Sonoma St,2121.0,42.308718,-71.087375,0101000020E6100000AA7BDD8B97C551C05453E50E8427...,Citizens Connect App,NaT,0,12,NaN
2724580,101005114367,2023-10-16 19:04:17,2023-10-18 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,INTERSECTION Minot St & Hallet St,NaN,42.285705,-71.046982,0101000020E6100000DB48E1BE01C351C0C1BBC7FB9124...,Citizens Connect App,NaT,0,16,NaN
2724579,101005114366,2023-10-16 18:49:34,2023-11-15 17:49:36,NaT,ONTIME,Open,,Rodent Activity,Inspectional Services,Environmental Services,...,108 Jersey St,2215.0,42.343230,-71.096931,0101000020E61000002FBE431C34C651C0728855F2EE2B...,Citizens Connect App,NaT,0,5,NaN
2724578,101005114363,2023-10-16 18:44:28,2023-11-15 17:44:30,NaT,ONTIME,Open,,Rodent Activity,Inspectional Services,Environmental Services,...,61 Chestnut St,2114.0,42.357185,-71.069697,0101000020E6100000FD9B65EC75C451C0D5900C39B82D...,Citizens Connect App,NaT,0,5,NaN
2724577,101005114361,2023-10-16 18:41:15,2023-10-24 04:30:00,NaT,ONTIME,Open,,Ground Maintenance,Parks & Recreation Department,Park Maintenance & Safety,...,45 Prince St,2113.0,42.364843,-71.054661,0101000020E610000023710E8F7FC351C0E7504829B32E...,Citizens Connect App,NaT,0,3,NaN
2724576,101005114360,2023-10-16 18:41:00,2023-11-15 17:41:10,NaT,ONTIME,Open,,No Utilities Residential - Water,Inspectional Services,Housing,...,70-90 Ames Way,2124.0,42.289519,-71.086148,0101000020E6100000D851597183C551C0C2BD49F40E25...,Constituent Call,NaT,0,14,NaN


In [ ]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [ ]:
clean_data.head()

,case_enquiry_id,queue_ANML02_LostFound,queue_ANML_General,queue_BHA_General,queue_BPD_Administrative,queue_BPD_Districts,queue_BPS_Administrative,queue_BPS_Transportation Administration,queue_BTDT_AVRS Interface Queue,queue_BTDT_Abandoned Bicycle,...,location_zipcode_2136.0,location_zipcode_2163.0,location_zipcode_2199.0,location_zipcode_2201.0,location_zipcode_2203.0,location_zipcode_2210.0,location_zipcode_2215.0,location_zipcode_2445.0,location_zipcode_2446.0,location_zipcode_2467.0
0,101005074111,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,101005074113,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,101005074114,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,101005074116,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,101005074118,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
mydata = clean_data

In [ ]:
mydata['case_enquiry_id']

0        101005074111
1        101005074113
2        101005074114
3        101005074116
4        101005074118
             ...     
21959    101005114370
21960    101005114373
21961    101005114376
21962    101005114377
21963    101005114379
Name: case_enquiry_id, Length: 21964, dtype: object

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        X[col] = X[col].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [ ]:
#concatenate the two dataframes and reindex
df = X

In [ ]:
df.shape

(279547, 7)

In [ ]:


# Assuming df is your DataFrame and it has columns 'desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding'
desc_cls_embedding_flattened = np.stack(df['desc_cls_embedding'].to_numpy())
desc_pooled_embedding_flattened = np.stack(df['desc_pooled_embedding'].to_numpy())
#do the same for the rest
name_cls_embedding_flattened = np.stack(df['name_cls_embedding'].to_numpy())
name_pooled_embedding_flattened = np.stack(df['name_pooled_embedding'].to_numpy())
code_cls_embedding_flattened = np.stack(df['code_cls_embedding'].to_numpy())
code_pooled_embedding_flattened = np.stack(df['code_pooled_embedding'].to_numpy())


# Remove the old columns
df.drop(['desc_cls_embedding', 'desc_pooled_embedding'], axis=1, inplace=True)
#do the same for the rest
df.drop(['name_cls_embedding', 'name_pooled_embedding'], axis=1, inplace=True)
df.drop(['code_cls_embedding', 'code_pooled_embedding'], axis=1, inplace=True)


# Add the new flattened columns
df_desc_cls = pd.DataFrame(desc_cls_embedding_flattened, columns=[f'desc_cls_{i}' for i in range(desc_cls_embedding_flattened.shape[1])])

df_desc_pooled = pd.DataFrame(desc_pooled_embedding_flattened, columns=[f'desc_pooled_{i}' for i in range(desc_pooled_embedding_flattened.shape[1])])
#do the same for the rest
df_name_cls = pd.DataFrame(name_cls_embedding_flattened, columns=[f'name_cls_{i}' for i in range(name_cls_embedding_flattened.shape[1])])
df_name_pooled = pd.DataFrame(name_pooled_embedding_flattened, columns=[f'name_pooled_{i}' for i in range(name_pooled_embedding_flattened.shape[1])])
df_code_cls = pd.DataFrame(code_cls_embedding_flattened, columns=[f'code_cls_{i}' for i in range(code_cls_embedding_flattened.shape[1])])
df_code_pooled = pd.DataFrame(code_pooled_embedding_flattened, columns=[f'code_pooled_{i}' for i in range(code_pooled_embedding_flattened.shape[1])])



df = pd.concat([df, df_desc_cls, df_desc_pooled, df_name_cls, df_name_pooled, df_code_cls, df_code_pooled], axis=1)

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [ ]:
df = df[is_numeric]

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [ ]:
df.shape

(279547, 769)

In [ ]:
mydata.shape

(21964, 785)

In [ ]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [ ]:
new_mydata.shape

(14467, 1553)

In [ ]:

df = new_mydata

In [ ]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [ ]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
code_pooled_123             float64
code_pooled_124             float64
code_pooled_125             float64
code_pooled_126             float64
code_pooled_127             float64
Length: 1553, dtype: object


In [ ]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [ ]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [ ]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

  1/453 [..............................] - ETA: 23s

2023-10-21 00:47:31.338910: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.339657: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.344244: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.344955: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.351770: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.448018: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.448964: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.449553: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31.451173: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:31

453/453 [==============================] - 1s 1ms/step


2023-10-21 00:47:32.154841: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.155665: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.226775: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.232320: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.232895: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.233882: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.
2023-10-21 00:47:32.242594: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:508] ROCm Fusion is enabled.


In [ ]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.7613005 0.0969354 0.032076877 0.021694522 0.017461317 0.012811556 0.013169362 0.0015464094 0.007566911 0.009391227 0.00551551 0.0036490096 0.0074261464 0.0037833562 0.0006806318 0.00048762155 0.00030200102 0.0004058931 0.00037116482 0.0002809067 0.00020614968 0.0002175961 0.00023563427 0.00021791965 7.646738e-05 0.00014991417 0.00012513666 0.00011111348 0.00010836753 4.511326e-05 7.7814955e-05 5.5340115e-05 0.0001520425 4.079423e-05 8.143171e-05 7.870383e-05 6.521878e-05 5.2311585e-05 6.280552e-05 0.00013733486 3.2899377e-05 3.4541736e-05 2.7484684e-05 6.3846295e-05 4.1013027e-05 4.374375e-05 5.5204328e-05 2.4416726e-05 4.4245455e-05 3.7279202e-05 2.2876506e-05 3.6212656e-05 2.6859627e-05 6.100873e-05 4.3824963e-05 2.9796012e-05 1.706698e-05 2.74025e-05 5.3632615e-05 1.6258438e-05 0.00010676429'
 '0.32553586 0.10922436 0.061913274 0.042108495 0.026823152 0.022408541 0.02168497 0.016116066 0.013726756 0.01359467 0.011748581 0.012381467 0.011471144 0.011847741 0.00889171 0.007007027 

In [ ]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [ ]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [ ]:
len(predictions_df)

14467

In [ ]:
print(predictions)

[[7.6130050e-01 9.6935399e-02 3.2076877e-02 ... 5.3632615e-05
  1.6258438e-05 1.0676429e-04]
 [3.2553586e-01 1.0922436e-01 6.1913274e-02 ... 3.4009900e-03
  2.6514041e-03 6.7017205e-02]
 [9.9369305e-01 4.8298850e-03 4.6718275e-04 ... 8.1011294e-06
  1.0718090e-06 1.6154892e-04]
 ...
 [9.9484551e-01 3.7206155e-03 4.0354097e-04 ... 6.3221055e-06
  6.3307846e-07 8.0723548e-05]
 [8.3300257e-01 8.2916155e-02 2.3019550e-02 ... 1.6387827e-05
  4.4667654e-06 3.6667512e-05]
 [2.0463717e-01 1.3968723e-01 1.0356064e-01 ... 3.2710468e-03
  3.8819273e-03 4.7915809e-02]]


In [ ]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [ ]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20231021_003909_Boston311KerasNLP  Boston311KerasNLP    2023-10-21


In [ ]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [ ]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [ ]:
all_model_predictions['ml_model_name'] = model_name

In [ ]:
all_model_predictions['prediction_date'] = today_datestring

In [ ]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [ ]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [ ]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-21_00-47-32_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-21_00-47-32_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-21_00-47-32_manifest.txt u353344964@195.179.236.61:/

65002

In [ ]:
# %%
if copy_to_prod:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-21_00-47-32_manifest.txt

Total records to process: 36432

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-21_00-47-32_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 21864.
Total records remaining: 36332.
Time for last 100 records: 0.02 seconds.
Estimated time remaining for this file: 4 seconds.
Estimated time for all files: 7 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 21764.
Total records remaining: 36232.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 seconds.
Estimated time for all files: 4 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 21664.
Total records remaining: 36132.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 se